In [ ]:
import pandas as pd # Librería para analizar los datos

In [ ]:
# Carga el archivo CSV
data_heartrate_seconds = pd.read_csv("Fitabase Data 4.12.16-5.12.16/heartrate_seconds_merged.csv")
print(data_heartrate_seconds.head())

In [ ]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Muestra el número de valores NaN en cada columna
data_heartrate_seconds.isna().sum()

In [ ]:
from sklearn.impute import SimpleImputer
mode_imputer = SimpleImputer(strategy = 'most_frequent') # moda

# Hacer inputación por cada columna
for column in data_heartrate_seconds.columns:
    values = data_heartrate_seconds[column].values.reshape(-1,1)
    mode_imputer.fit(values)
    data_heartrate_seconds[column] = mode_imputer.transform(values)

# Comprobamos que hayamos eliminadolos atípicos o erroneos.
data_heartrate_seconds.isna().sum()

In [ ]:
# Convierte la columna de fecha a formato datetime
data_heartrate_seconds['Time'] = pd.to_datetime(data_heartrate_seconds['Time'])

# Agrupa por ID y fecha redondeada al minuto, calcula la media y reinicia el índice
result_df = data_heartrate_seconds.groupby(['Id', pd.Grouper(key='Time', freq='1Min')]).mean().reset_index()

# Guarda el resultado en un nuevo archivo CSV
result_df.to_csv("Fitabase Data 4.12.16-5.12.16/heartrate_minutes_merged.csv", index=False)

print()

In [ ]:
# Carga el archivo CSV
data_intensities_minutes = pd.read_csv("Fitabase Data 4.12.16-5.12.16/minuteIntensitiesNarrow_merged.csv")
print(data_intensities_minutes.head())
# Muestra el número de valores NaN en cada columna
data_intensities_minutes.isna().sum()

from sklearn.impute import SimpleImputer
mode_imputer = SimpleImputer(strategy = 'most_frequent') # moda

# Hacer inputación por cada columna
for column in data_intensities_minutes.columns:
    values = data_intensities_minutes[column].values.reshape(-1,1)
    mode_imputer.fit(values)
    data_intensities_minutes[column] = mode_imputer.transform(values)

# Comprobamos que hayamos eliminadolos atípicos o erroneos.
data_intensities_minutes.isna().sum()

In [ ]:
# Carga el archivo CSV
#data_dailyActivity_merged = pd.read_csv("Fitabase Data 4.12.16-5.12.16/dailyActivity_merged.csv")
#print(data_intensities_minutes.head())
# Muestra el número de valores NaN en cada columna
#data_intensities_minutes.isna().sum()

#from sklearn.impute import SimpleImputer
#mode_imputer = SimpleImputer(strategy = 'most_frequent') # moda

# Hacer inputación por cada columna
#for column in data_intensities_minutes.columns:
 #   values = data_intensities_minutes[column].values.reshape(-1,1)
  #  mode_imputer.fit(values)
  #  data_intensities_minutes[column] = mode_imputer.transform(values)

# Comprobamos que hayamos eliminadolos atípicos o erroneos.
#data_intensities_minutes.isna().sum()

Sleep Enum 1->Asleep 2-->Restless 3-->Awake

In [ ]:
# Carga el archivo CSV
data_minutes_sleep = pd.read_csv("Fitabase Data 4.12.16-5.12.16/minuteSleep_merged.csv")
print(data_minutes_sleep.head())
# Muestra el número de valores NaN en cada columna
data_minutes_sleep.isna().sum()

from sklearn.impute import SimpleImputer
mode_imputer = SimpleImputer(strategy = 'most_frequent') # moda

# Hacer inputación por cada columna
for column in data_minutes_sleep.columns:
    values = data_minutes_sleep[column].values.reshape(-1,1)
    mode_imputer.fit(values)
    data_minutes_sleep[column] = mode_imputer.transform(values)

# Comprobamos que hayamos eliminadolos atípicos o erroneos.
data_minutes_sleep.isna().sum()

In [ ]:

# Convertir la columna 'date' a tipo datetime
data_minutes_sleep['date'] = pd.to_datetime(data_minutes_sleep['date'])

# Restar 30 segundos a cada entrada
data_minutes_sleep['date'] = data_minutes_sleep['date'] - pd.Timedelta(seconds=30)

# Ajustar minutos y segundos a 00
data_minutes_sleep['date'] = data_minutes_sleep['date'].dt.floor('T')

# Guardar el nuevo dataset en un nuevo archivo CSV
#data_minutes_sleep.to_csv("minuteSleep_merged.csv", index=False)

# Elimina la columna 'LogId'
data_minutes_sleep = data_minutes_sleep.drop(columns=['logId'])

# Guarda el DataFrame modificado de nuevo en el archivo CSV original
data_minutes_sleep.to_csv("minuteSleep_merged.csv", index=False)

print("La columna 'LogId' se ha eliminado exitosamente.")



In [ ]:
print(data_minutes_sleep)

A continuacion imprimeros todos los dataset, leidos hasta el momento

In [ ]:
#cambiar el nombre de la variable data del csv heartrate por uno mas explicativo
print("Ahora imprimeros la pulsaciones por minito (heartrate_seconds_merged.csv.csv) \n")
print(result_df.head())

print("\n")

print("Ahora imprimeros las intensidades por minuto (data_intensities_minutes.csv) \n")
print(data_intensities_minutes.head())

print("\n")
print("Ahora imprimeros la cabecera de (minuteSleep_merged.csv)")
print(data_minutes_sleep.head())


In [ ]:
# Convertir la columna 'Time' a tipo datetime en data_intensities_minutes
data_intensities_minutes['ActivityMinute'] = pd.to_datetime(data_intensities_minutes['ActivityMinute'])

# Fusionar los DataFrames utilizando 'Id' y 'Time'
merged_df = pd.merge(result_df, data_intensities_minutes, left_on=['Id', 'Time'], right_on=['Id', 'ActivityMinute'])

# Seleccionar las columnas deseadas y renombrarlas
result_df = merged_df[['Id', 'Time', 'Value', 'Intensity']]
result_df.columns = ['Id', 'Time', 'HeartRate', 'Intensity']

# Imprimir las pulsaciones por minuto
print("Pulsaciones por minuto:")
print(result_df[['Id', 'Time', 'HeartRate']])

# Imprimir las intensidades por minuto
print("\nIntensidades por minuto:")
print(result_df[['Id', 'Time', 'Intensity']])

In [ ]:
# Guardar el nuevo dataset en un nuevo archivo CSV
result_df.to_csv("Fitabase Data 4.12.16-5.12.16/test_train_data.csv", index=False)

In [ ]:
data_minutes_sleep.isna().sum()

A continuación, procederemos con el entrenamiento del los distintos modelos utilizados:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Extraemos características de la columna 'Time'
result_df['Hour'] = result_df['Time'].dt.hour
result_df['Weekday'] = result_df['Time'].dt.weekday

# Definir las características y la variable objetivo
features = ['Id', 'Hour', 'Weekday', 'Intensity']
target = 'HeartRate'

# Separar las características y la variable objetivo
X = result_df[features]
y = result_df[target]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=11101)

# Crear un transformador para manejar las variables categoricas
categorical_features = ['Id']
numeric_features = ['Hour', 'Weekday', 'Intensity']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Crear el modelo de Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=11101)

# Crear el pipeline que combina el preprocesamiento y el modelo
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

# Entrenar el modelo
pipeline.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de prueba
score = pipeline.score(X_test, y_test)

print(f'R2 Score on Test Set: {score}')

